In [56]:
link = 'D:/users/Marko/downloads/mirna/'

# Imports

In [57]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [58]:
import sys
#sys.path.insert(0,'/content/drive/MyDrive/Marko/master')
sys.path.insert(0, link)
import numpy as np
import matplotlib.pyplot as plt

#import tensorflow as tf

import torch
import torch.optim as optim
import torch.nn as nn
import torch.distributions as dist

from torch.nn import functional as F
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import OneHotEncoder

from tqdm import tqdm
from tqdm import trange

import datetime


writer = SummaryWriter(f"{link}/saved_models/new/IVAE/tensorboard")

In [59]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
DEVICE

device(type='cuda')

# Model Classes

In [61]:
class diva_args:

    def __init__(self, z_dim=128, d_dim=45, x_dim=7500, y_dim=2,
                 beta=10, rec_alpha = 1, rec_beta = 1, 
                 rec_gamma = 1, warmup = 1, prewarmup = 1):

        self.z_dim = z_dim
        self.d_dim = d_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        
        self.beta = beta
        self.rec_alpha = rec_alpha
        self.rec_beta = rec_beta
        self.rec_gamma = rec_gamma
        self.warmup = warmup
        self.prewarmup = prewarmup


## Dataset Class

In [62]:
class MicroRNADataset(Dataset):

    def __init__(self, ds='train', create_encodings=False, use_subset=False):
        
        # loading images
        self.images = np.load(f'{link}/data/modmirbase_{ds}_images.npz')['arr_0']/255
        
        
        # loading labels
        print('Loading Labels! (~10s)')     
        ohe = OneHotEncoder(categories='auto', sparse=False)
        labels = np.load(f'{link}/data/modmirbase_{ds}_labels.npz')['arr_0']
        self.labels = ohe.fit_transform(labels)
        
        # loading encoded images
        print("loading encodings")
        if create_encodings:
            x_len, x_bar, x_col = self.get_encoded_values(self.images, ds)
        else:
            x_len = np.load(f'{link}/data/modmirbase_{ds}_images_len_new.npz')
            x_bar = np.load(f'{link}/data/modmirbase_{ds}_images_bar_new.npz')
            x_col = np.load(f'{link}/data/modmirbase_{ds}_images_col_new.npz')
        
        self.x_len = x_len
        self.x_bar = x_bar
        self.x_col = x_col
        
        
        self.mountain = np.load(f'{link}/data/modmirbase_{ds}_mountain.npy')
        
        
        # loading names
        print('Loading Names! (~5s)')
        names =  np.load(f'{link}/data/modmirbase_{ds}_names.npz')['arr_0']
        names = [i.decode('utf-8') for i in names]
        self.species = ['mmu', 'prd', 'hsa', 'ptr', 'efu', 'cbn', 'gma', 'pma',
                        'cel', 'gga', 'ipu', 'ptc', 'mdo', 'cgr', 'bta', 'cin', 
                        'ppy', 'ssc', 'ath', 'cfa', 'osa', 'mtr', 'gra', 'mml',
                        'stu', 'bdi', 'rno', 'oan', 'dre', 'aca', 'eca', 'chi',
                        'bmo', 'ggo', 'aly', 'dps', 'mdm', 'ame', 'ppc', 'ssa',
                        'ppt', 'tca', 'dme', 'sbi']
        # assigning a species label to each observation from species
        # with more than 200 observations from past research
        self.names = []
        for i in names:
            append = False
            for j in self.species:
                if j in i.lower():
                    self.names.append(j)
                    append = True
                    break
            if not append:
                if 'random' in i.lower() or i.isdigit():
                    self.names.append('hsa')
                else:
                    self.names.append('notfound')
        
        # performing one hot encoding
        ohe = OneHotEncoder(categories='auto', sparse=False)
        
       
        
        self.names_ohe = ohe.fit_transform(np.array(self.names).reshape(-1,1))
          
        if use_subset:    
            idxes = [i == 'hsa' and np.random.choice([True, False]) for i in self.names]
            self.names_ohe = self.names_ohe[idxes]
            self.labels = self.labels[idxes]
            self.images = self.images[idxes]
            self.x_len = self.x_len[idxes]
            self.x_col = self.x_col[idxes]
            self.x_bar = self.x_bar[idxes]
            self.mountain = self.mountain[idxes]
    
    def __len__(self):
        return(self.images.shape[0])

    def __getitem__(self, idx):
        d = self.names_ohe[idx]
        y = self.labels[idx]
        x = self.images[idx]
        x = np.transpose(x, (2,0,1))
        x_len = self.x_len[idx]
        x_col = self.x_col[idx]
        x_bar = self.x_bar[idx]
        mount = self.mountain[idx]                        
        return (x, y, d, x_len, x_col, x_bar, mount)


    def get_encoded_values(self, x, ds):
        """
        given an image or batch of images
        returns length of strand, length of bars and colors of bars
        """
        n = x.shape[0]
        x = np.transpose(x, (0,3,1,2))
        out_len = np.zeros((n), dtype=np.uint8)
        out_col = np.zeros((n,26,100), dtype=np.uint8)
        out_bar = np.zeros((n,2,100), dtype=np.uint8)

        for i in range(n):
            if i % 100 == 0:
                print(f'at {i} out of {n}')
            rna_len = 0
            broke = False
            for j in range(100):
                if (x[i,:,12,j] == np.array([1,1,1])).all():
                    out_len[i] = rna_len
                    broke = True
                    out_col[i,25,j] = 1
                else:
                    rna_len += 1
                    # check color of bars
                    out_col[i, self.get_color(x,i,j),j] = 1 
                    #out_col[i, self.get_color(x[i,:,13,j]), 1, j] = 1
                    # check length of bars
                    len1 = 0
                    # loop until white pixel
                    while not (x[i,:,12-len1,j] == np.array([1.,1.,1.])).all():
                        len1 += 1
                        if 13-len1 == 0:
                            break
                    out_bar[i, 0, j] = len1

                    len2 = 0
                    while not (x[i,:,13+len2,j] == np.array([1.,1.,1.])).all():
                        len2 += 1
                        if 13+len2 == 25:
                            break
                    out_bar[i, 1, j] = len2
            if not broke:
                out_len[i] = rna_len


        with open(f'{link}/data/modmirbase_{ds}_images_len_new.npz', 'wb') as f:
            np.save(f, out_len)
        with open(f'{link}/data/modmirbase_{ds}_images_col_new.npz', 'wb') as f:
            np.save(f, out_col)
        with open(f'{link}/data/modmirbase_{ds}_images_bar_new.npz', 'wb') as f:
            np.save(f, out_bar)
        

        return out_len, out_bar, out_col

    def get_color(self, x, i, j):
        
        col = self._get_color(x[i,:,12,j])+self._get_color(x[i,:,13,j])
        if col == '00':
            return 0
        if col == '01':
            return 1
        if col == '02':
            return 2
        if col == '03':
            return 3
        if col == '04':
            return 4
        if col == '10':
            return 5
        if col == '11':
            return 6
        if col == '12':
            return 7
        if col == '13':
            return 8
        if col == '14':
            return 9
        if col == '20':
            return 10
        if col == '21':
            return 11
        if col == '22':
            return 12
        if col == '23':
            return 13
        if col == '24':
            return 14
        if col == '30':
            return 15
        if col == '31':
            return 16
        if col == '32':
            return 17
        if col == '33':
            return 18
        if col == '34':
            return 19
        if col == '40':
            return 20
        if col == '41':
            return 21
        if col == '42':
            return 22
        if col == '43':
            return 23
        if col == '44':
            return 24
        
        
    
    def _get_color(self, pixel):
        """
        returns the encoded value for a pixel
        """
        if (pixel == np.array([0,0,0])).all():  
            return "0" # black
        elif (pixel == np.array([1,0,0])).all():  
            return "1" # red
        elif (pixel == np.array([0,0,1])).all():  
            return "2" # blue
        elif (pixel == np.array([0,1,0])).all():  
            return "3" # green
        elif (pixel == np.array([1,1,0])).all():  
            return "4" # yellow
        else:
            print("Something wrong!")


## Decoder classes

In [63]:
# Decoders
class px(nn.Module):
    def __init__(self, d_dim, x_dim, y_dim, z_dim, dim1=256, dim2=512):
        super(px, self).__init__()

        self.fc = nn.Sequential(nn.Linear(z_dim, dim1, bias=False),  
                                 nn.ReLU(),
                                nn.Linear(dim1, dim2),
                                nn.ReLU())
        
        # Predicting length and color of each bar
        
        self.color = nn.Sequential(nn.Linear(dim2, 2600))
        
        
        self.length_bar = nn.Sequential(nn.Linear(dim2,200), nn.Softplus())
        
        
    def forward(self, z):
        
        h = self.fc(z)
        
        
        
        len_bar = self.length_bar(h).reshape(-1,2,100)
        len_bar_sc = nn.Parameter(torch.tensor([1.])).to(DEVICE)
        
        
        col = self.color(h).reshape(-1,26,100)
        col_bar = nn.Softmax(dim=1)(col)
        
        return len_bar, len_bar_sc, col_bar

    def reconstruct_image(self, len_bar, var_bar ,col_bar, sample=False):
        """
        reconstructs RNA image given output from decoder
        even indexes of len_bar and col_bar   -> top
        uneven indexes of len_bar and col_bar -> bottom
        function does not support sampling yet
        color reconstructions: 0: black
                               1: red
                               2: blue
                               3: green
                               4: yellow
        """
        color_dict = {
                  0: np.array([0,0,0]), # black
                  1: np.array([1,0,0]), # red
                  3: np.array([0,1,0]), # green
                  2: np.array([0,0,1]), # blue
                  4: np.array([1,1,0]),  # yellow
                  5: np.array([1,1,1])  # white
                  }
    
        _color_dict =  {0: (0,0),
                        1: (0,1),
                        2: (0,2),
                        3: (0,3),
                        4: (0,4),
                        5: (1,0),
                        6: (1,1),
                        7: (1,2),
                        8: (1,3),
                        9: (1,4),
                        10: (1,0),
                        11: (2,1),
                        12: (2,2),
                        13: (2,3),
                        14: (2,4),
                        15: (2,0),
                        16: (3,1),
                        17: (3,2),
                        18: (3,3),
                        19: (3,4),
                        20: (3,0),
                        21: (4,1),
                        22: (4,2),
                        23: (4,3),
                        24: (4,4),
                        25: (5,5)
                        }       
        len_bar = len_bar.cpu().numpy()
        var_bar = var_bar.cpu().numpy()
        col_bar = col_bar.cpu().numpy()
        n = len_bar.shape[0]
        output = np.ones((n,25,100,3))

        for i in range(n):
            limit = 100
            for j in range(limit):
                if sample:
                    _len_bar_1 = int(np.round(np.random.normal(loc=len_bar[i,0,j], scale=var_bar[i,0,j])))
                    _len_bar_2 = int(np.round(np.random.normal(loc=len_bar[i,1,j], scale=var_bar[i,1,j])))
                    _col_bar_1 = np.random.choice(np.arange(5), p = col_bar[i, :, 2*j])
                    _col_bar_2 = np.random.choice(np.arange(5), p = col_bar[i,:, 2*j+1])
                else:
                    _len_bar_1 = int(np.round(len_bar[i,0,j])) 
                    _len_bar_2 = int(np.round(len_bar[i,1,j]))
                    _col_bar_1, _col_bar_2 = _color_dict[np.argmax(col_bar[i,:,j])]
                    
                h1 = max(0,13-_len_bar_1)
                # paint upper bar
                output[i, h1:13, j] = color_dict[_col_bar_1]
                h2 = min(25,13+_len_bar_2)
                # paint lower bar
                output[i, 13:h2, j] = color_dict[_col_bar_2]
        
        
        return output

In [64]:
int(np.round(3.7, 0))
int(3.7)

3

In [65]:
# pzy_ = pzy(45, 7500, 2, 32,32,32)
# summary(pzy_, (1,2))
# pzy_ = px(45, 7500, 2, 32,32,32)
# summary(pzy_, [(1,32),(1,32),(1,32)])

## Endcoder Classes

In [66]:
#pzy_.reconstruct_image(torch.zeros((1,100)), torch.zeros((1,13,200)), torch.zeros(1,5,200)).shape

In [67]:
class inception_color(nn.Module):
    def __init__(self, filters):
        super(inception_color, self).__init__()
        
        self.filters = filters
        
        self.color_tower = nn.Sequential(
            nn.Conv2d(3, 1, kernel_size=1, stride=1, padding = 'same'),
            nn.ReLU(),
            nn.MaxPool2d((13,1),stride=(13,1), ceil_mode=True)
            
        )
        self.shape_tower = nn.Sequential(
            nn.Conv2d(3, 12, kernel_size=1, stride=1),
            nn.ReLU(),
            nn.ZeroPad2d((0,0,0,1)), # shape -1,26,100,12
            nn.Conv2d(12, self.filters, kernel_size=(13,5), stride=(13,1)),
            nn.ReLU())



    def forward(self, x):
        col = self.color_tower(x)
        col = col.view(-1,200)
        shp = self.shape_tower(x)
        shp = shp.view(-1, 2*96*self.filters)

        out = torch.cat([col,shp],1)
        return out

In [68]:
class inception_A(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=16, out_channels=32):
        super(inception_A, self).__init__()
        
        self.tower_1 = nn.Sequential(
            nn.AvgPool2d((3,3), stride=1, padding=1, count_include_pad=False),
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        
        self.tower_2 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        
        self.tower_3 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU())
        
        self.tower_3a = nn.Sequential(
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(7,3), padding='same'),
            nn.ReLU()
        )
        
        self.tower_3b = nn.Sequential(
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(3,7), padding='same'),
            nn.ReLU()
        )
        
        self.tower_4 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=(1,1), padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(3,7), padding='same'),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=(7,3), padding='same'),
            nn.ReLU()
        )
        
        self.out = nn.ReLU()
        
    def forward(self, x):
        t1 = self.tower_1(x)
        t2 = self.tower_2(x)
        t3 = self.tower_3(x)
        ta = self.tower_3a(t3)
        tb = self.tower_3b(t3)
        t4 = self.tower_4(x)
        cat = torch.cat([t1,t2,ta,tb,t4],1)
        out = self.out(cat)
        return out
    
    
class inception_B(nn.Module):
    def __init__(self, in_channels=128, hidden_channels=64, out_channels=128):
        super(inception_B, self).__init__()
        
        self.tower_1 = nn.Sequential(
            nn.AvgPool2d((3,3), stride=1, padding=1, count_include_pad=False),
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        
        self.tower_2 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        
        self.tower_3 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(1,7), padding='same'),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=(1,7), padding='same'),
            nn.ReLU(),
            
            
        )
        
        
        self.tower_4 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=(1,1), padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=(1,7), padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(7,1), padding='same'),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=(1,7), padding='same'),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=(7,1), padding='same'),
            nn.ReLU()
        )
        
        self.out = nn.ReLU()
        
    def forward(self, x):
        t1 = self.tower_1(x)
        t2 = self.tower_2(x)
        t3 = self.tower_3(x)
        t4 = self.tower_4(x)
        cat = torch.cat([t1,t2,t3,t4],1)
        out = self.out(cat)
        return out
    
class inception_C(nn.Module):
    def __init__(self, in_channels=128, hidden_channels=64, out_channels=128):
        super(inception_C, self).__init__()
        
        self.tower_1 = nn.Sequential(
            nn.AvgPool2d((3,3), stride=1, padding=1, count_include_pad=False),
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        
        self.tower_2 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU()
        )
        
        self.tower_3 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU())
        
        self.tower_3a = nn.Sequential(
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(5,1), padding='same'),
            nn.ReLU()
        )
        
        self.tower_3b = nn.Sequential(
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(1,5), padding='same'),
            nn.ReLU()
        )
        
        
        self.tower_4 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=(1,1), padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=(1,3), padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=(3,1), padding='same'),
            nn.ReLU(),
        )
        
        self.tower_4a = nn.Sequential(
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(3,1), padding='same'),
            nn.ReLU()
        )
        
        self.tower_4b = nn.Sequential(
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(1,3), padding='same'),
            nn.ReLU()
        )
        
        self.out = nn.ReLU()
        
    def forward(self, x):
        t1 = self.tower_1(x)
        t2 = self.tower_2(x)
        t3 = self.tower_3(x)
        t3a = self.tower_3a(t3)
        t3b = self.tower_3b(t3)
        t4 = self.tower_4(x)
        t4a = self.tower_4a(t4)
        t4b = self.tower_4b(t4)
        cat = torch.cat([t1,t2,t3a,t3b,t4a,t4b],1)
        out = self.out(cat)
        return out

In [69]:
class reduction_A(nn.Module):
    def __init__(self, in_channels=128, hidden_channels=128, out_channels=128):
        super(reduction_A, self).__init__()
        
        self.tower_1 = nn.Sequential(
            nn.MaxPool2d((3,7), stride=(2,3)),
        )
        
        self.tower_2 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=(3,7), stride=(2,3), padding='valid'),
            nn.ReLU()
        )
        
        self.tower_3 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(3,7), stride=(2,3), padding='valid'),
            nn.ReLU(),
        )
        
        self.out = nn.ReLU()
        
    def forward(self, x):
        t1 = self.tower_1(x)
        t2 = self.tower_2(x)
        t3 = self.tower_3(x)
        cat = torch.cat([t1,t2,t3],1)
        out = self.out(cat)
        return out
    
    
class reduction_B(nn.Module):
    def __init__(self, in_channels=384, hidden_channels=64, out_channels=128):
        super(reduction_B, self).__init__()
        
        self.tower_1 = nn.Sequential(
            nn.MaxPool2d((3,7), stride=(2,3)),
        )
        
        self.tower_2 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=(3,7), stride=(2,3), padding='valid'),
            nn.ReLU()
        )
        
        self.tower_3 = nn.Sequential(
            nn.Conv2d(in_channels, hidden_channels, kernel_size=1, padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=(1,7), padding='same'),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, out_channels, kernel_size=(7,1), padding='same'),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=(3,7), stride=(2,3), padding='valid'),
            nn.ReLU(),
        )
        
        self.out = nn.ReLU()
        
    def forward(self, x):
        t1 = self.tower_1(x)
        t2 = self.tower_2(x)
        t3 = self.tower_3(x)
        cat = torch.cat([t1,t2,t3],1)
        out = self.out(cat)
        return out

In [70]:
inc = inception_color(4)
summary(inc, (1,3,25,100))

D:\Users\Marko\Anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Layer (type:depth-idx)                   Output Shape              Param #
inception_color                          --                        --
├─Sequential: 1-1                        [1, 1, 2, 100]            --
│    └─Conv2d: 2-1                       [1, 1, 25, 100]           4
│    └─ReLU: 2-2                         [1, 1, 25, 100]           --
│    └─MaxPool2d: 2-3                    [1, 1, 2, 100]            --
├─Sequential: 1-2                        [1, 4, 2, 96]             --
│    └─Conv2d: 2-4                       [1, 12, 25, 100]          48
│    └─ReLU: 2-5                         [1, 12, 25, 100]          --
│    └─ZeroPad2d: 2-6                    [1, 12, 26, 100]          --
│    └─Conv2d: 2-7                       [1, 4, 2, 96]             3,124
│    └─ReLU: 2-8                         [1, 4, 2, 96]             --
Total params: 3,176
Trainable params: 3,176
Non-trainable params: 0
Total mult-adds (M): 0.73
Input size (MB): 0.03
Forward/backward pass size (MB): 0.27

In [71]:
inc = inc.to(DEVICE)

In [72]:
inc(torch.zeros((1,3,25,100)).to(DEVICE)).shape

torch.Size([1, 968])

In [73]:
inc = inception_A(3)
summary(inc, (1,3,25,100))

Layer (type:depth-idx)                   Output Shape              Param #
inception_A                              --                        --
├─Sequential: 1-1                        [1, 16, 25, 100]          --
│    └─AvgPool2d: 2-1                    [1, 3, 25, 100]           --
│    └─Conv2d: 2-2                       [1, 16, 25, 100]          64
│    └─ReLU: 2-3                         [1, 16, 25, 100]          --
├─Sequential: 1-2                        [1, 16, 25, 100]          --
│    └─Conv2d: 2-4                       [1, 16, 25, 100]          64
│    └─ReLU: 2-5                         [1, 16, 25, 100]          --
├─Sequential: 1-3                        [1, 16, 25, 100]          --
│    └─Conv2d: 2-6                       [1, 16, 25, 100]          64
│    └─ReLU: 2-7                         [1, 16, 25, 100]          --
├─Sequential: 1-4                        [1, 32, 25, 100]          --
│    └─Conv2d: 2-8                       [1, 32, 25, 100]          10,784
│    └─ReLU

In [74]:
inc = reduction_A(256)
summary(inc, (1,256,25,100))

Layer (type:depth-idx)                   Output Shape              Param #
reduction_A                              --                        --
├─Sequential: 1-1                        [1, 256, 12, 32]          --
│    └─MaxPool2d: 2-1                    [1, 256, 12, 32]          --
├─Sequential: 1-2                        [1, 128, 12, 32]          --
│    └─Conv2d: 2-2                       [1, 128, 12, 32]          688,256
│    └─ReLU: 2-3                         [1, 128, 12, 32]          --
├─Sequential: 1-3                        [1, 128, 12, 32]          --
│    └─Conv2d: 2-4                       [1, 128, 25, 100]         32,896
│    └─ReLU: 2-5                         [1, 128, 25, 100]         --
│    └─Conv2d: 2-6                       [1, 128, 25, 100]         147,584
│    └─ReLU: 2-7                         [1, 128, 25, 100]         --
│    └─Conv2d: 2-8                       [1, 128, 12, 32]          344,192
│    └─ReLU: 2-9                         [1, 128, 12, 32]         

In [75]:
inc = inception_B(512)
summary(inc, (1,512,12,32))

Layer (type:depth-idx)                   Output Shape              Param #
inception_B                              --                        --
├─Sequential: 1-1                        [1, 64, 12, 32]           --
│    └─AvgPool2d: 2-1                    [1, 512, 12, 32]          --
│    └─Conv2d: 2-2                       [1, 64, 12, 32]           32,832
│    └─ReLU: 2-3                         [1, 64, 12, 32]           --
├─Sequential: 1-2                        [1, 64, 12, 32]           --
│    └─Conv2d: 2-4                       [1, 64, 12, 32]           32,832
│    └─ReLU: 2-5                         [1, 64, 12, 32]           --
├─Sequential: 1-3                        [1, 128, 12, 32]          --
│    └─Conv2d: 2-6                       [1, 64, 12, 32]           32,832
│    └─ReLU: 2-7                         [1, 64, 12, 32]           --
│    └─Conv2d: 2-8                       [1, 128, 12, 32]          57,472
│    └─ReLU: 2-9                         [1, 128, 12, 32]          --

In [76]:
inc = reduction_B(384)
summary(inc, (1,384,12,32))

Layer (type:depth-idx)                   Output Shape              Param #
reduction_B                              --                        --
├─Sequential: 1-1                        [1, 384, 5, 9]            --
│    └─MaxPool2d: 2-1                    [1, 384, 5, 9]            --
├─Sequential: 1-2                        [1, 64, 5, 9]             --
│    └─Conv2d: 2-2                       [1, 64, 12, 32]           24,640
│    └─ReLU: 2-3                         [1, 64, 12, 32]           --
│    └─Conv2d: 2-4                       [1, 64, 5, 9]             86,080
│    └─ReLU: 2-5                         [1, 64, 5, 9]             --
├─Sequential: 1-3                        [1, 128, 5, 9]            --
│    └─Conv2d: 2-6                       [1, 64, 12, 32]           24,640
│    └─ReLU: 2-7                         [1, 64, 12, 32]           --
│    └─Conv2d: 2-8                       [1, 64, 12, 32]           28,736
│    └─ReLU: 2-9                         [1, 64, 12, 32]           --

In [77]:
inc = inception_C(576)
summary(inc, (1,576,5,9))

Layer (type:depth-idx)                   Output Shape              Param #
inception_C                              --                        --
├─Sequential: 1-1                        [1, 64, 5, 9]             --
│    └─AvgPool2d: 2-1                    [1, 576, 5, 9]            --
│    └─Conv2d: 2-2                       [1, 64, 5, 9]             36,928
│    └─ReLU: 2-3                         [1, 64, 5, 9]             --
├─Sequential: 1-2                        [1, 64, 5, 9]             --
│    └─Conv2d: 2-4                       [1, 64, 5, 9]             36,928
│    └─ReLU: 2-5                         [1, 64, 5, 9]             --
├─Sequential: 1-3                        [1, 64, 5, 9]             --
│    └─Conv2d: 2-6                       [1, 64, 5, 9]             36,928
│    └─ReLU: 2-7                         [1, 64, 5, 9]             --
├─Sequential: 1-4                        [1, 128, 5, 9]            --
│    └─Conv2d: 2-8                       [1, 128, 5, 9]            41,088

In [78]:
class qz(nn.Module):
    def __init__(self, d_dim, x_dim, y_dim, z_dim):
        super(qz, self).__init__()

        self.inc_A = nn.Sequential( 
            inception_A(3, 8, 8),
            inception_A(40,16,16),
        )
        self.red_A = nn.Sequential(
            reduction_A(80,16,16)
        )
        self.inc_B = nn.Sequential(
            inception_B(112,16,16)
        )
        self.red_B = nn.Sequential(
            reduction_B(64,16,16)
        )
        self.inc_C = nn.Sequential(
            inception_C(96,64,96),
            #inception_C(512,32,64),
            nn.MaxPool2d(2,2)
        )
        
        self.inc_COL = inception_color(4)
        
        self.fc = nn.Sequential(nn.Linear(4096, 512), nn.ReLU())
        
        self.z_mu = nn.Sequential(nn.Linear(512+968, z_dim))
        self.z_si = nn.Sequential(nn.Linear(512+968, z_dim), nn.Softplus())


    def forward(self, x):
        h = self.inc_A(x)
        h = self.red_A(h)
        h = self.inc_B(h)
        h = self.red_B(h)
        h = self.inc_C(h)
        h = h.view(-1,4096)
        h = self.fc(h)
        
        c = self.inc_COL(x)
        ch = torch.cat([c,h],1)
        z_loc = self.z_mu(ch)
        z_scale = self.z_si(ch) + 1e-7

        return z_loc, z_scale




In [79]:
enc = qz(128,10,10,1024)
summary(enc, (1,3,25,100))

Layer (type:depth-idx)                   Output Shape              Param #
qz                                       --                        --
├─Sequential: 1-1                        [1, 80, 25, 100]          --
│    └─inception_A: 2-1                  [1, 40, 25, 100]          --
│    │    └─Sequential: 3-1              [1, 8, 25, 100]           32
│    │    └─Sequential: 3-2              [1, 8, 25, 100]           32
│    │    └─Sequential: 3-3              [1, 8, 25, 100]           32
│    │    └─Sequential: 3-4              [1, 8, 25, 100]           1,352
│    │    └─Sequential: 3-5              [1, 8, 25, 100]           1,352
│    │    └─Sequential: 3-6              [1, 8, 25, 100]           2,736
│    │    └─ReLU: 3-7                    [1, 40, 25, 100]          --
│    └─inception_A: 2-2                  [1, 80, 25, 100]          --
│    │    └─Sequential: 3-8              [1, 16, 25, 100]          656
│    │    └─Sequential: 3-9              [1, 16, 25, 100]          656
│   

## Full model class

In [80]:
class StampDIVA(nn.Module):
    def __init__(self, args):
        super(StampDIVA, self).__init__()
        self.z_dim = args.z_dim
        self.d_dim = args.d_dim
        self.x_dim = args.x_dim
        self.y_dim = args.y_dim

        self.px = px(self.d_dim, self.x_dim, self.y_dim, self.z_dim)
        
        self.qz = qz(self.d_dim, self.x_dim, self.y_dim, self.z_dim)
        

        self.beta = args.beta
        
        self.rec_alpha = args.rec_alpha
        self.rec_beta = args.rec_beta
        self.rec_gamma = args.rec_gamma

        self.warmup = args.warmup
        self.prewarmup = args.prewarmup

        self.cuda()

    def forward(self, d, x, y):
        # Encode
        zd_q_loc, zd_q_scale = self.qz(x)
        
        # Reparameterization trick
        qz = dist.Normal(zd_q_loc, zd_q_scale)
        z_q = qz.rsample()
        
        
        # Decode
        x_bar, x_bar_scale, x_col = self.px(z_q)
        z_p_loc, z_p_scale = torch.zeros(z_q.size()[0], self.z_dim).cuda(),\
                        torch.ones(z_q.size()[0], self.z_dim).cuda()
        pz = dist.Normal(z_p_loc, z_p_scale)

        # Reparameterization trick
        pz = dist.Normal(z_p_loc, z_p_scale)
        
        return x_bar, x_bar_scale, x_col, qz, pz, z_q

    def loss_function(self, d, x, y, out_len, out_bar, out_col):
        
        x_bar, x_bar_scale, x_col, qz, pz, z_q = self.forward(d, x, y)
       
        mse_bar = (((x_bar - out_bar)**2)).mean(dim=(1,2)).sum()
        
        max_bar = torch.argmax(x_col, dim=1)
        acc_bar = (max_bar==torch.argmax(out_col, dim=1)).float().mean((1)).sum().float()
        
        CE_bar = mse_bar#-log_bar
        CE_col = F.cross_entropy(x_col, out_col, reduction='sum')

        KL_z = torch.sum(pz.log_prob(z_q) - qz.log_prob(z_q))
          
        return self.rec_beta * CE_bar \
                  + self.rec_gamma * CE_col \
                  - self.beta * KL_z, \
                  CE_bar, CE_col, mse_bar, acc_bar

# Training the model

## Loading dataset

In [81]:
RNA_dataset = MicroRNADataset(create_encodings=False)

Loading Labels! (~10s)
loading encodings
Loading Names! (~5s)


In [82]:
RNA_dataset_test = MicroRNADataset('test', create_encodings=False)

Loading Labels! (~10s)
loading encodings
Loading Names! (~5s)


In [83]:
len(RNA_dataset)

34721

In [84]:
def train_single_epoch(train_loader, model, optimizer, epoch):
    model.train()
    train_loss = 0
    epoch_bar_loss = 0
    epoch_col_loss = 0
    no_batches = 0
    mse_bar = 0
    acc_bar = 0
    pbar = tqdm(enumerate(train_loader), unit="batch", 
                                     desc=f'Epoch {epoch}')
    for batch_idx, (x, y, d, x_len, x_col, x_bar,_) in pbar:
        # To device
        x, y, d , x_len, x_bar, x_col = x.to(DEVICE), y.to(DEVICE), d.to(DEVICE), x_len.to(DEVICE), x_bar.to(DEVICE), x_col.to(DEVICE)

        optimizer.zero_grad()
        loss, bar_loss, col_loss, mse, acc = model.loss_function(d.float(), x.float(), y.float(), x_len.float(), x_bar.float(), x_col.float())
      
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item()/x.shape[0])
        train_loss += loss
        epoch_bar_loss += bar_loss
        epoch_col_loss += col_loss
        mse_bar += mse
        acc_bar += acc
        no_batches += 1

    train_loss /= len(train_loader.dataset)
    epoch_bar_loss /= len(train_loader.dataset)
    epoch_col_loss /= len(train_loader.dataset)
    acc_bar /= len(train_loader.dataset)
    mse_bar /= len(train_loader.dataset)
    
    return train_loss, epoch_bar_loss, epoch_col_loss, mse_bar, acc_bar

In [85]:
def test_single_epoch(test_loader, model, epoch):
    model.eval()
    test_loss = 0
    epoch_bar_loss = 0
    epoch_col_loss = 0
    mse_bar = 0
    acc_bar = 0        
    with torch.no_grad():
        for batch_idx, (x,y,d,x_len,x_col,x_bar,_) in enumerate(test_loader):
            x, y, d, x_len, x_bar, x_col = x.to(DEVICE), y.to(DEVICE), d.to(DEVICE), x_len.to(DEVICE), x_bar.to(DEVICE), x_col.to(DEVICE)
            loss, bar_loss, col_loss, mse, acc = model.loss_function(d.float(), x.float(), y.float(),x_len.float(),x_bar.float(),x_col.float())
            test_loss += loss
            epoch_bar_loss += bar_loss
            epoch_col_loss += col_loss
            mse_bar += mse
            acc_bar += acc
    test_loss /= len(test_loader.dataset)
    epoch_bar_loss /= len(test_loader.dataset)
    epoch_col_loss /= len(test_loader.dataset)
    acc_bar /= len(test_loader.dataset)
    mse_bar /= len(test_loader.dataset)
    
    return test_loss, epoch_bar_loss, epoch_col_loss, mse_bar, acc_bar
  

In [86]:
def train(args, train_loader, test_loader, diva, optimizer, end_epoch, start_epoch=0, save_folder='sd_1.0.0',save_interval=5):
    
    epoch_loss_sup = []
    test_loss = []
    
    for epoch in range(start_epoch+1, end_epoch+1):
        diva.beta = min([args.beta, args.beta * (epoch - args.prewarmup * 1.) / (args.warmup)])
        if epoch< args.prewarmup:
            diva.beta = args.beta/args.prewarmup
        train_loss, avg_loss_bar, avg_loss_col, mtr, atr = train_single_epoch(train_loader, diva, optimizer, epoch)
        str_loss_sup = train_loss
        epoch_loss_sup.append(train_loss)
        str_print = "epoch {}: avg train loss {:.2f}".format(epoch, str_loss_sup)
        str_print += ", bar train loss {:.3f}".format(avg_loss_bar)
        str_print += ", col train loss {:.3f}".format(avg_loss_col)
        print(str_print)

        rec_loss_train = diva.rec_beta * avg_loss_bar + diva.rec_gamma * avg_loss_col
        dis_loss_train = train_loss - rec_loss_train

        test_lss, avg_loss_bar_test, avg_loss_col_test, mte, ate = test_single_epoch(test_loader, diva, epoch)
        test_loss.append(test_lss)
       
        str_print = "epoch {}: avg test  loss {:.2f}".format(epoch, test_lss)
        str_print += ", bar  test loss {:.3f}".format(avg_loss_bar_test)
        str_print += ", col  test loss {:.3f}".format(avg_loss_col_test)
        print(str_print)

        rec_loss_test = diva.rec_beta * avg_loss_bar_test + diva.rec_gamma * avg_loss_col_test
        dis_loss_test = test_lss - rec_loss_test

        if writer is not None:
            
            writer.add_scalars("Total_Loss", {'train': train_loss, 'test': test_lss} ,epoch)
            writer.add_scalars("Reconstruction_vs_Disentanglement",{'rec':rec_loss_train, 'dis':dis_loss_train}, epoch)
            writer.add_scalars("bar_mse",{'train': mtr, 'test':mte}, epoch)
            writer.add_scalars("bar_acc",{'train': atr, 'test':ate}, epoch)

        if epoch % save_interval == 0:
            save_reconstructions(epoch, test_loader, diva, name=save_folder)
            save_reconstructions(epoch, train_loader, diva, name=save_folder, estr='tr')
        
        
        if epoch % 50 == 0:
            torch.save(diva.state_dict(), f'{link}/saved_models/{save_folder}/checkpoints/{epoch}.pth')

    if writer is not None:
        writer.flush()

    epoch_loss_sup = [i.detach().cpu().numpy() for i in epoch_loss_sup]
    test_loss = [i.detach().cpu().numpy() for i in test_loss]
    return epoch_loss_sup, test_loss

In [87]:
def save_reconstructions(epoch, test_loader, diva, name='diva', estr=''):
    a = next(enumerate(test_loader))
    with torch.no_grad():
        diva.eval()
        d = a[1][2][:10].to(DEVICE).float()
        x = a[1][0][:10].to(DEVICE).float()
        y = a[1][1][:10].to(DEVICE).float()
        m = a[1][-1][:10].to(DEVICE).float()
        x_2, x_2var, x_3 ,qz, pz, z_q = diva(d,x,y)
        out = diva.px.reconstruct_image(x_2, x_2var, x_3)

    plt.figure(figsize=(80,20))
    fig, ax = plt.subplots(nrows=10, ncols=2)

    ax[0,0].set_title("Original")
    ax[0,1].set_title("Reconstructed")

    for i in range(10):
        ax[i, 1].imshow(out[i])
        ax[i, 0].imshow(x[i].cpu().permute(1,2,0))
        ax[i, 0].xaxis.set_visible(False)
        ax[i, 0].yaxis.set_visible(False)
        ax[i, 1].xaxis.set_visible(False)
        ax[i, 1].yaxis.set_visible(False)
    fig.tight_layout(pad=0.1)
    plt.savefig(f'{link}/saved_models/{name}/reconstructions/e{epoch}{estr}.png')
    plt.close('all')

In [88]:
DEVICE

device(type='cuda')

## Model Training

In [89]:
default_args = diva_args(z_dim=1024, rec_alpha = 20, rec_beta = 10, rec_gamma = 10, 
                         beta=1, warmup=1, prewarmup=0)

In [90]:
diva = StampDIVA(default_args).to(DEVICE)

In [91]:
#diva.load_state_dict(torch.load(f'{link}/saved_models/VAE10/checkpoints/905.pth'))

In [92]:
train_loader = DataLoader(RNA_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(RNA_dataset_test, batch_size=128)

In [93]:
#optimizer = optim.SGD(diva.parameters(), lr=0.00001, momentum=0.1, nesterov=True)
optimizer = optim.Adam(diva.parameters(), lr=0.005)

In [94]:
RNA_dataset.x_len.min(), RNA_dataset.x_len.max()

(10, 100)

In [95]:
writer.flush()

In [96]:
%tensorboard --logdir="D:/users/Marko/downloads/mirna/saved_models/new/IVAE/tensorboard/"

Reusing TensorBoard on port 6006 (pid 2580), started 23:21:55 ago. (Use '!kill 2580' to kill it.)

In [97]:
lss, lss_t = train(default_args, train_loader, test_loader, diva, optimizer, 500, 0, save_folder="new/IVAE",save_interval=5)

Epoch 1: 0batch [00:00, ?batch/s]D:\Users\Marko\Anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
Epoch 1: 272batch [01:10,  3.84batch/s, loss=2.89e+3]


epoch 1: avg train loss 2940.47, bar train loss 9.602, col train loss 281.537
epoch 1: avg test  loss 2895.62, bar  test loss 8.081, col  test loss 281.103


Epoch 2: 272batch [01:04,  4.24batch/s, loss=2.9e+3] 


epoch 2: avg train loss 2892.75, bar train loss 7.987, col train loss 281.085


Epoch 3: 0batch [00:00, ?batch/s]

...

epoch 4: avg train loss 2890.87, bar train loss 7.920, col train loss 281.062


Epoch 5: 0batch [00:00, ?batch/s]

epoch 4: avg test  loss 2890.62, bar  test loss 7.925, col  test loss 281.042


Epoch 5: 53batch [00:12,  4.13batch/s, loss=2.91e+3]


KeyboardInterrupt: 

In [ ]:
lss2, lss_t2 = train(default_args, train_loader, test_loader, diva, optimizer, 1000, 500, save_folder="new/IVAE")

In [ ]:
lss, lss_t = train(default_args, train_loader, test_loader, diva, optimizer, 1600, 1000, save_folder="new/IVAE")

In [ ]:
def plot_loss_acc(lss, lss_t):
    fig,ax = plt.subplots()
    ax.plot(lss, label="train loss")
    ax.plot(lss_t, label = "test loss")
    #ax1 = ax.twinx()
    #ax1.plot(yacc, label = "train accuracy", ls='--')
    #ax1.plot(yacc_t, label = "test accuracy", ls='--')

    lines, labels = ax.get_legend_handles_labels()
    #lines2, labels2 = ax1.get_legend_handles_labels()

    ax.legend(lines, labels)

In [ ]:
plot_loss_acc(lss, lss_t)

In [ ]:
plot_loss_acc(lss3, lss_t3, yacc3, yacc_t3)

In [ ]:
def plot_change_latent_var(diva, lat_space="y", var_idx=[0,1,2,3,4,5,6,7], step = 5):
    a = next(enumerate(test_loader))
    with torch.no_grad():
        diva.eval()
        d = a[1][2][:len(var_idx)].to(DEVICE).float()
        x = a[1][0][:len(var_idx)].to(DEVICE).float()
        y = a[1][1][:len(var_idx)].to(DEVICE).float()

        zx, zx_sc = diva.qzx(x)
        zy, zy_sc = diva.qzy(x)
        zd, zd_sc =  diva.qzd(x)

        print(torch.max(zy), torch.min(zy), "sdmax:", torch.max(zy_sc))

        out = change(zx, zy, zd, var_idx, lat_space, diva, step)
    
    fig, ax = plt.subplots(ncols=out.shape[0],nrows=len(var_idx),figsize=(10*4*out.shape[0],10*len(var_idx)))
    for i in range(out.shape[0]):
      for j in range(len(var_idx)):
        ax[j,i].imshow(out[i,j])

In [ ]:
def change(zx, zy, zd, idx, lat = "y", model=diva, step = 2):
    
    dif = np.arange(-30,15,step)
    print(torch.max(zy), torch.min(zy))
    out = np.zeros((dif.shape[0], len(idx), 25, 100 ,3))  
    #print(zy.shape, dif.shape[0])
    for i in range(dif.shape[0]):
      for j in range(len(idx)):
        if lat == "y":
            zy[j,idx] = dif[i]
        elif lat == "x":
            zx[j,idx] = dif[i]
        elif lat == "d":
            zd[j,idx] = dif[i]
        len_, bar, col = model.px(zd[j],zx[j],zy[j])
        out[i,j] = model.px.reconstruct_image(len_[None,:], bar, col)
    
    return out



In [ ]:
plot_change_latent_var(diva)

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(50,120), [i.cpu().detach().numpy() for i in lss2], label="train loss")
ax.plot(np.arange(50,120), [i.cpu().detach().numpy() for i in lss_t2], label = "testloss")
ax1 = ax.twinx()
ax1.plot(np.arange(50,120), yacc2, label = "train")
ax1.plot(np.arange(50,120), yacc_t2, label = "test")

plt.legend()

In [ ]:
fig,ax = plt.subplots()
ax.plot(np.arange(120,180), [i.cpu().detach().numpy() for i in lss3], label="train loss")
ax.plot(np.arange(120,180), [i.cpu().detach().numpy() for i in lss_t3], label = "testloss")
ax1 = ax.twinx()
ax1.plot(np.arange(120,180), yacc3, label = "train",c='green')
ax1.plot(np.arange(120,180), yacc_t3, label = "test")

plt.legend()

# Model Evaluation

## Sampling from trained model

In [ ]:
def plot_latent_space(lat_space="y"):
    '''
    lat_space: y, d, x
    '''

    

In [ ]:
plot(x, out, 0)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3)
for i in range(9):
  ax[i//3, i%3].imshow(x[i].cpu().permute(1,2,0))
  
plt.savefig('divastamporg.png')